In [1]:
"""
実行時に設定するパラメータ
"""

# Informerの学習パラメータ
# Informerを学習するときに使用したパラメータの文字列をここで代入する
ARG_STR = "--model informer --data ETTh1 --attn prob --freq h --checkpoints 'checkpoints/ETTh1_sample30_window30' --features S  --e_layers 1  --d_layers 1 --dropout 0.3 --learning_rate 0.0001 --embed timeF"
# ARG_STR = "--model informer --data NaturalGas --root_path './Informer2020/data/NaturalGas/' --data_path combined_data.csv --features S --attn prob --freq h --checkpoints 'checkpoints/NaturalGas_sample30_window30' --e_layers 1  --d_layers 1 --dropout 0.3 --learning_rate 0.0001 --embed timeF"

# 学習済みのInformerモデルが保存されているパス
INFORMER_CKPT_PATH = "checkpoints/naturalgas.pth"

In [2]:
import sys

sys.path.append("Informer2020")

import argparse
import random
import tqdm
import numpy as np
import torch
import pandas as pd
import os

import dataset
from model.informer_model import InformerModel
from model.model import Model
from model.moment_model import MomentModel
from propose import ProposedModelWithMoe, ProposedModel
from evaluation import evaluate_mse, evaluate_nll

from main_informer import parse_args

In [3]:
def set_seed(seed: int) -> None:
    # random
    random.seed(seed)

    # numpy
    np.random.seed(seed)

    # pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.mps.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


def load_moment_model(args: argparse.Namespace) -> Model:
    return MomentModel(param="AutonLab/MOMENT-1-large", pred_len=args.pred_len)


def load_moment_model_finetuned(
    args: argparse.Namespace,
    train_dataset: torch.utils.data.Dataset,
    valid_dataset: torch.utils.data.Dataset,
) -> Model:
    model: MomentModel = load_moment_model(args=args)
    model.fine_tuning(
        train_dataset=train_dataset, valid_dataset=valid_dataset, args=args
    )
    return model


def load_informer_model(args: argparse.Namespace) -> Model:
    return InformerModel(args, checkpoint_path=INFORMER_CKPT_PATH)


def load_proposed_model(moment_model: Model, informer_model: Model) -> Model:
    model = ProposedModel(moment_model=moment_model, informer_model=informer_model)
    return model


def load_proposed_model_with_moe(
    moment_model: Model,
    informer_model: Model,
    input_size: int,
    train_dataset: torch.utils.data.Dataset,
    valid_dataset: torch.utils.data.Dataset,
    args: argparse.Namespace,
) -> Model:
    model = ProposedModelWithMoe(
        moment_model=moment_model,
        informer_model=informer_model,
        input_size=input_size,
    )
    model.train(train_dataset=train_dataset, valid_dataset=valid_dataset, args=args)
    return model

In [4]:
set_seed(0)

args = parse_args(ARG_STR)
save_file_name = args.checkpoints
if not os.path.exists(save_file_name):
    os.mkdir(save_file_name)
print("args:", args)

train_dataset, valid_dataset, test_dataset = dataset.load_dataset(args=args)
input_size = args.seq_len

moment_model = load_moment_model(args=args)
informer_model = load_informer_model(args=args)
proposed_model = load_proposed_model(moment_model, informer_model)

args: Namespace(model='informer', data='ETTh1', root_path='./data/ETT/', data_path='ETTh1.csv', features='S', target='OT', freq='h', checkpoints='checkpoints/ETTh1_sample30_window30', seq_len=96, label_len=48, pred_len=24, enc_in=1, dec_in=1, c_out=1, d_model=512, n_heads=8, e_layers=1, d_layers=1, s_layers=[3, 2, 1], d_ff=2048, factor=5, padding=0, distil=True, dropout=0.3, attn='prob', embed='timeF', activation='gelu', output_attention=False, do_predict=False, mix=True, cols=None, num_workers=0, itr=2, train_epochs=6, batch_size=32, patience=3, learning_rate=0.0001, des='test', loss='mse', lradj='type1', use_amp=False, inverse=False, use_gpu=False, gpu=0, use_multi_gpu=False, devices='0,1,2,3', detail_freq='h')
train 8521
val 2857
test 2857


In [ ]:
proposed_model_with_moe = load_proposed_model_with_moe(
    moment_model, informer_model, input_size, train_dataset, valid_dataset, args
)

In [6]:
# 予測結果の評価
print("args:", args)
results = {}

for method, model in {
    "informer": informer_model,
    "moment": moment_model,
    "proposed": proposed_model,
    "proposed+moe": None,
}.items():
    print(f"testing: {method}")
    test_dataloader = dataset.to_dataloader(test_dataset, args, "test")

    y_pred = []
    y_true = []
    for index, batch in tqdm.tqdm(test_dataloader):
        y_pred.append(model.predict_distr(index, batch).detach().tolist())
        y_true.append(batch[1][:, -1].squeeze().detach().tolist())
        break

    y_pred, y_true = np.array(y_pred).reshape(-1, 2), np.array(y_true).flatten()
    results[method] = {
        "mse": evaluate_mse(y_pred, y_true),
        "nll": evaluate_nll(y_pred, y_true),
    }
    print(results[method])

results

args: Namespace(model='informer', data='ETTh1', root_path='./data/ETT/', data_path='ETTh1.csv', features='S', target='OT', freq='h', checkpoints='checkpoints/ETTh1_sample30_window30', seq_len=96, label_len=48, pred_len=24, enc_in=1, dec_in=1, c_out=1, d_model=512, n_heads=8, e_layers=1, d_layers=1, s_layers=[3, 2, 1], d_ff=2048, factor=5, padding=0, distil=True, dropout=0.3, attn='prob', embed='timeF', activation='gelu', output_attention=False, do_predict=False, mix=True, cols=None, num_workers=0, itr=2, train_epochs=6, batch_size=2, patience=3, learning_rate=0.0001, des='test', loss='mse', lradj='type1', use_amp=False, inverse=False, use_gpu=False, gpu=0, use_multi_gpu=False, devices='0,1,2,3', detail_freq='h')
testing: informer


  0%|          | 0/1428 [00:00<?, ?it/s]

2
[]
[0, 1]


  0%|          | 0/1428 [00:04<?, ?it/s]


{'mse': 2.4509563667488012, 'nll': 3542520.571500522}
testing: moment


...

  0%|          | 0/1428 [00:03<?, ?it/s]


{'mse': 0.01657331083171773, 'nll': 11.826302061459845}
testing: proposed


  0%|          | 0/1428 [00:00<?, ?it/s]

0
[0, 1]
[0, 1]
0
[0, 1]
[0, 1]
{'mse': 0.7175221952768116, 'nll': 2305.376903813798}


{'informer': {'mse': 2.4509563667488012, 'nll': 3542520.571500522},
 'moment': {'mse': 0.01657331083171773, 'nll': 11.826302061459845},
 'proposed': {'mse': 0.7175221952768116, 'nll': 2305.376903813798}}

In [7]:
pd.DataFrame(results).to_csv("results.csv", index=False)